# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [40]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
link_system_prompt += "here is additionnal example: "
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about", was_a_relative_path: true},
        {"type": "careers page": "url": "https://another.full.url/careers", was_a_relative_path: false}
    ]
}
"""

In [41]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
here is additionnal example: 
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about", was_a_relative_path: true},
        {"type": "careers page": "url": "https://another.full.url/careers", was_a_relative_path: false}
    ]
}



In [49]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "if a link was a relative links, in the json indicate it to true otherwize don't don't add that key:\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [44]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
if a link was a relative links, in the json indicate it to true. and do not add it if not:
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner

In [59]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [57]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/meta-llama/Llama-3.3-70B-Instruct',
 '/tencent/HunyuanVideo',
 '/Datou1111/shou_xin',
 '/black-forest-labs/FLUX.1-dev',
 '/deepseek-ai/DeepSeek-V2.5-1210',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/multimodalart/flux-style-shaping',
 '/spaces/ginipick/FLUXllama',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/CohereForAI/Global-MMLU',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/amphora/QwQ-LongCoT-130K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers

In [60]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page',
   'url': 'https://huggingface.co/huggingface',
   'was_a_relative_path': True},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page',
   'url': 'https://huggingface.co/enterprise',
   'was_a_relative_path': True},
  {'type': 'pricing page',
   'url': 'https://huggingface.co/pricing',
   'was_a_relative_path': True},
  {'type': 'blog',
   'url': 'https://huggingface.co/blog',
   'was_a_relative_path': True},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [20]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links::", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [21]:
print(get_all_details("https://huggingface.co"))

Found links:: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on model

In [63]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [65]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += f"The Title must be bold and subtitle in italic\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [32]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links:: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmeta-llama/Llama-3.3-70B-Instruct\nUpdated\n3 days ago\n•\n102k\n•\n960\ntencent/HunyuanVideo\nUpdated\n6 days ago\n•\n3.73k\n•\n992\nDatou1111/shou_xin\nUpdated\n5 days ago\n•\n7.84k\n•\n322\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16\n•\n1.38M\n•\n7.23k\nQwen/QwQ-32B-Preview\nUpdated\n15 days ago\n•\n92.8k\n•\n1.27k\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.13k\n🏢\nTRELLIS\nScalable and Versatile 3D Generation from ima

In [67]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    return response.choices[0].message.content
   

In [71]:
brochure= create_brochure("HuggingFace", "https://huggingface.com")
display(Markdown(brochure))

Found links:: {'links': [{'type': 'about page', 'url': 'https://huggingface.com'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'company page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# **Hugging Face**
### _The AI Community Building the Future_

---

## 💡 About Us
Hugging Face is not just a company; it's a vibrant community at the forefront of artificial intelligence and machine learning. Our platform serves as a collaborative space where enthusiasts, researchers, and developers come together to create, discover, and share cutting-edge models, datasets, and applications. With a commitment to open source and innovation, we're shaping the future of AI.

## 🌍 Our Offerings
- **Models**: Explore over 400,000 machine learning models, including state-of-the-art architectures for various tasks across text, image, video, and audio domains.
- **Datasets**: Access a rich repository of 100,000+ datasets to enhance your ML projects.
- **Spaces**: Deploy and share your applications seamlessly on our platform, fostering collaboration and discovery.
- **Enterprise Solutions**: We provide tailored solutions with enterprise-grade security, support, and access controls for over 50,000 organizations, including notable giants like Google, Microsoft, and Amazon Web Services.

## 🤝 Our Customers
We cater to a diverse range of clients, from emerging startups to established enterprises looking to leverage AI capabilities. Hugging Face is trusted by organizations like Meta, Grammarly, and Intel, making it a go-to platform for machine learning advancement.

## 🌱 Company Culture
At Hugging Face, we foster an inclusive and dynamic culture that encourages creativity and collaboration. Our team is composed of enthusiastic individuals from various backgrounds who share a passion for artificial intelligence. We believe in the power of community contributions to make machine learning accessible to everyone. A commitment to continuous learning and support for individual growth encourages employees to innovate and challenge the status quo.

## 🚀 Careers at Hugging Face
Join us in our mission to democratize AI! We are on the lookout for talented individuals across various domains including engineering, data science, and community engagement. If you are passionate about building tools that impact the future of AI, consider applying for a position. Explore our current job openings on our [Jobs page](https://huggingface.co/jobs).

## 🌟 Why Choose Hugging Face?
- **Open Source Commitment**: We believe in the power of open collaboration and sharing knowledge.
- **Innovative Tools**: Our state-of-the-art tools and libraries streamline the ML development process.
- **Community Focus**: We thrive on contributions from our community, ensuring the best resources are at everyone's fingertips.

---

For more information, visit our website at [HuggingFace.co](https://huggingface.co) and join us in building the future of AI!
```

In [82]:
def translate(content, lang):
    print("start_the_translation")
    trans_system_prompt = f"Has an assistant, you have the task to translate in {lang} the content and keep the result in markdown"
    trans_user_prompt = "You received the content of the brochure. you must translate it for international client\n."
    trans_user_prompt += f"The content is:{content}"
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":trans_system_prompt },
            {"role": "user", "content": trans_user_prompt}
          ],
    )
    return response.choices[0].message.content

In [85]:
display(Markdown(translate(brochure, "lingala")))

start_the_translation


```markdown
# **Hugging Face**
### _Communauté ya AI oyo ezalaka na etape ya koleka_

---

## 💡 Ekomeli
Hugging Face ezali te pekeli; ezali communauté ya makasi na ntango ya kitoko ya intelijans ya makasi mpe kokanga boye. Plateforme na biso ezali ndako ya bokokani epayi ya ba-enthousiasts, ba-researchers, mpe ba-developers oyo bakotaki kosepela, kobongisa, mpe koyamba ba-modèles, ba-datasets, mpe ba-applications ya malamu. Na ekateli ya kosala na makambo ya sika mpe bokeseni, tozali kolongola avenir ya AI.

## 🌍 Makambo na biso
- **Ba-Modèles**: Talá bisika ya ba-modèles ya machine learning koleka 400,000, okata na ba-architectures ya kitoko mpo na makambo ebele na domaine ya texte, image, vidéo, mpe audio.
- **Ba-Datasets**: Pesa nzela na ba-datasets ya 100,000+ mpo na kobundisa misala na yo ya ML.
- **Ba-Spaces**: Lula mpe koyamba ba-applications na yo na nzela ya kitoko na plateforme na biso, kosembola bokokani mpe kokanisa.
- **Soluté ya Entreprise**: Tozali kopesa makambo oyo etali entreprise na bokamba ya sécurité, soutien, mpe contrôles ya accès mpo na ba-organisation koleka 50,000, na ba-grands comme Google, Microsoft, mpe Amazon Web Services.

## 🤝 Ba-Clients na biso
Tosala na ba-clients nyonso, na banzela ya ba-startups ya sika ti na ba-entreprises ya kala oyo balingaka kokanisa makambo ya AI. Hugging Face ezali site ya boko ya ba-organisation lokola Meta, Grammarly, mpe Intel, oyo esalelaka yango mpo na ntango ya kokola ya machine learning.

## 🌱 Culture ya Entreprise
Na Hugging Face, tosalaka culture ya kokamwa mpe kolanda molende oyo ebotaka bokokani mpe bokeseni. Équipe na biso ezali na ba-personnes ya gete oyo bazali na makambo ebele ya bokinaka, kasi bokeseni ya intelijans ya makasi. Tozali na likanisi ya makasi mpo na basani ya communauté mpo na komibanzela machine learning mpo na nyonso. État ya kotalela mpo na koyekola mpo na bokeseni ya moto moko na moko ekangani ba-employés na biso mpo na kolongola mabe mpe kosembola mpasi.

## 🚀 Ba-Carrières na Hugging Face
Landa biso na mosala na biso ya kolongola AI na komipesa! Tozali na ngai ya kotela ba-personnes ya makasi na ba-domaines ebele lokola engineering, data science, mpe engagement ya communauté. Soki ozali na likambo ya kolona ba-tools oyo ekoyeba avenir ya AI, zwela mosala. Talá ba-postes oyo ezali na [page ya Ba-Emplois](https://huggingface.co/jobs).

## 🌟 Mpo na nini Olingi Hugging Face?
- **État ya Ozala Makasi na Makambo ya Sika**: Tozali na likanisi ya makasi mpo na bokokani ya baninga mpe kondima koyamba makambo.
- **Ba-Tools ya Kitoko**: Ba-tools na biso mpe ba-archives ya kitoko basalaka makambo ya ml moko.
- **Koleka na Communauté**: Tokoki na sokisoko ya ba-contributions ya ba-communauté na biso, mpo na kokoba na nzela ya kitoko.

---

Mpo na makambo ebele, kende na website na biso na [HuggingFace.co](https://huggingface.co) mpe landa biso na kolongola avenir ya AI!
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [35]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [55]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links:: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forums', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# <span style="color:red">Welcome to Hugging Face!</span>

---

### 🤖 **The AI Community Helping You Hug It Out** 🤗

At Hugging Face, we don't just build AI; we build a whole community that dreams big and collaborates even bigger! Forget about flying solo; our platform offers a cozy corner where machine learning enthusiasts can come together, share models, datasets, and maybe even a virtual cup of coffee. ☕️

### 🚀 What’s Trending This Week?

- **meta-llama/Llama-3.3-70B-Instruct** - Updated 3 days ago and already has more followers than your favorite influencer: **102k**! 
- **tencent/HunyuanVideo** - Still waiting on a Netflix adaptation. Just **3.73k** users trying to make sense of it!
- **black-forest-labs/FLUX.1-dev** - Not just a coding project; it’s a whole experience—update rate: once a month and counting. **1.38M** supporters!

### 🏢 More Than Just Algorithms

We’re like a cozy library mixed with an explosion of creativity (and a sprinkle of caffeine)! With over **400,000 models** and **100,000 datasets**, you’ll find all the toys you need to build the ultimate machine learning castle! 🏰

### 🎉 **Join Our Hug-tastic Culture!**

- **Collaboration**: Work hand-in-hand with over **50,000 organizations** including big names like Google, Microsoft, and Grammarly! (Don’t worry, we don’t ask for autographs)
  
- **Open Source Fun**: Participate in creating cool new tools—like our Transformers™ library! Who knew a “transformer” could be cooler than a truck that turns into a robot? 🤔

### 🕵️‍♂️ Careers at Hugging Face

- **Looking for a Job?** We’re hiring! Take your skills from pretender to defender in the AI world. 
- **Perks**: Flexible workspace, competitive pay, and the option to take breaks every now and then to hug a co-worker! Just kidding, keep it professional…mostly! 😉

### 💌 Join Us Today!

Are you an AI aficionado or just really fond of hugging? 😅 Come for the algorithms, stay for the community! We provide paid Compute and Enterprise solutions, so whether you're a startup or a seasoned pro, we offer the right resources to transform your capabilities!

### 📞 Get In Touch!

Still have questions? Willing to engage in a deep dive discussion about whether machines can cry? Or just want to say hi? Check out our [website](https://huggingface.co) or follow us on social media.

---
### **Let’s Hug it Out With AI!**

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>